# AuxTel Calibration Illumination System monochromator slit test.

Craig Lage - 13Feb23

In [ ]:
import asyncio
import sys, time, os
from lsst.ts import utils

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from astropy.io import fits
from astropy.time import Time, TimeDelta
from astropy.table import Table

from lsst_efd_client import EfdClient
client = EfdClient('summit_efd')

from lsst.ts import salobj

In [ ]:
# change this to a local directory
data_dir = '/home/craiglagegit/DATA'

In [ ]:
d = salobj.Domain()

### Connect to CSCs

In [ ]:
atmonochromator = salobj.Remote(d, 'ATMonochromator')
FiberSpectrograph = salobj.Remote(name="FiberSpectrograph", domain=d, index=3)
electrometer = salobj.Remote(name="Electrometer", domain=d, index=201) 
WhiteLightSource = salobj.Remote(name='ATWhiteLight', domain=d)

In [ ]:
await atmonochromator.start_task
await FiberSpectrograph.start_task
await electrometer.start_task
await WhiteLightSource.start_task

In [ ]:
# Check that the monochromator has grating=0 and lambda=500
async def get_params():
    tmp1 = await atmonochromator.evt_wavelength.aget()
    tmp2 = await atmonochromator.evt_entrySlitWidth.aget()
    tmp3 = await atmonochromator.evt_exitSlitWidth.aget()
    tmp4 = await atmonochromator.evt_selectedGrating.aget()
    return (tmp1.wavelength, tmp2.width, tmp3.width, tmp4.gratingType)


# functions to run the electromter measurements
def get_file_transfer(lfa):
    filename = os.path.split(lfa.url)[1]
    save_dir = os.path.join(data_dir, filename.split(':')[0])
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
    save_file = os.path.join(save_dir, filename)
    os.system(f"curl {lfa.url} --output {save_file}")
    return save_file

async def elec_meas(exp_time):
    electrometer.evt_largeFileObjectAvailable.flush()
    tmp1=await electrometer.cmd_startScan.set_start(timeout=10)
    await asyncio.sleep(exp_time)
    tmp2=await electrometer.cmd_stopScan.set_start(timeout=10)
    lfa = await electrometer.evt_largeFileObjectAvailable.next(flush=False, timeout=10)
    filename = get_file_transfer(lfa)
    return filename, lfa

In [ ]:
# Run electrometer calibration
await electrometer.cmd_performZeroCalib.set_start(timeout=10)
await electrometer.cmd_setDigitalFilter.set_start(activateFilter=False, activateAvgFilter=False, activateMedFilter=False, timeout=10)    

In [ ]:
#Start chiller
tmp = await WhiteLightSource.cmd_setChillerTemperature.set_start(temperature=20)
tmp = await WhiteLightSource.cmd_startChiller.set_start()
# Confirm that the chiller has started and running at 20C

### Test Electrometer

In [ ]:
mode = await electrometer.evt_measureType.aget()
intTime = await electrometer.evt_integrationTime.aget()
range_ = await electrometer.evt_measureRange.aget()
print(f'Mode: {mode.mode}; Int. Time: {intTime.intTime}; Range: {range_.rangeValue}')

### Set up Monochromator

In [ ]:
wave=500
await atmonochromator.cmd_changeWavelength.set_start(wavelength=wave)
entry_width=7.0
exit_width=7.0
await atmonochromator.cmd_changeSlitWidth.set_start(slit=1, slitWidth=entry_width)
await atmonochromator.cmd_changeSlitWidth.set_start(slit=2, slitWidth=exit_width)

In [ ]:
await get_params()

In [ ]:
# Now run a matrix of slit widths
slit_readings = []
for i in range(1, 15):
    slit_1 = 0.5 * i
    await atmonochromator.cmd_changeSlitWidth.set_start(slit=1, slitWidth=slit_1)    
    for j in range(1, 15):
        slit_2 = 0.5 * j
        await atmonochromator.cmd_changeSlitWidth.set_start(slit=2, slitWidth=slit_2)    
        exp_time = 5 #sec
        filename, lfa = await elec_meas(5.0)
        filename = filename.replace(":", "_")
        elec_filen = os.path.join(data_dir, filename)
        os.system(f"curl {lfa.url} --output {elec_filen}")
        hdu = fits.open(f'{elec_filen}')
        data = hdu[1].data
        for (t,current) in data:
            slit_readings.append([t, slit_1, slit_2, -current])
        print(f"Done with {slit_1}, {slit_2}")


In [ ]:
# Pickle the data
import pickle as pkl
filename = "/home/craiglagegit/DATA/monochromator_slit_matrix_13feb23.pkl"
file = open(filename, "wb")
pkl.dump(slit_readings, file)
file.close()

In [ ]:
# Now plot the results
%matplotlib inline
fig = plt.figure(figsize=(8, 8))
slit_1s = np.arange(0.5, 7.5, 0.5)
slit_2s = np.arange(0.5, 7.5, 0.5)
Z = np.zeros([14,14])
[X,Y] = np.meshgrid(slit_1s, slit_2s);
    
for i, s1 in enumerate(slit_1s):
    for j, s2 in enumerate(slit_2s):
        counter = 0
        currentSum = 0.0
        for [t, slit_1, slit_2, current] in slit_readings:
            if (int(s1*10) == int(slit_1*10)) and (int(s2*10) == int(slit_2*10)):
                counter += 1
                currentSum += current
        currentAve = currentSum / float(counter)
        #print(f"{s1}, {s2} has {counter} points")
        X[i,j] = s1
        Y[i,j] = s2
        Z[i,j] = currentAve

img=plt.contourf(X,Y,Z)
plt.colorbar(img)                
plt.xlabel("Slit 1 width (mm)")
plt.ylabel("Slit 2 width (mm)")
plt.title("Electrometer current vs width of slits\n wavelength=500, grating 1")
plt.savefig(f"/home/craiglagegit/DATA/Monochromator_Slit_Test_13Feb23.pdf")